# On MongoDB

First, install mongo!

$\rightarrow$ You may need to adjust your security settings!

## Mongo in the Terminal

Let's try a few simple commands.

1. Run `mongo` to launch the program!
2. Now run `help` to see some mongo hints.
3. Let's run `show dbs` to show the databases to which we're currently connected.
4. To use or create a particular database, simply type `use` followed by the name of the database.
5. Once we're accessing a particular database, we can list its collections by running `show collections`.

But what's a collection? What are we doing with this Mongo thing? Let's talk about some concepts!

## What Is Mongo?

Mongo is one of the leading tools for working with *non-relational* databases.

With Mongo we should at least be able to Create, Read, Update, Delete: the four basic functions of persistent storage.

[This site](https://www.tutorialspoint.com/mongodb/index.htm) is an excellent resource on Mongo. Let's check it out!

- Overview: Terminology: SQL vs. NoSQL (Not only SQL)
- Advantages: NoSQL
- Data Modeling: Example
- Queries: Equivalents of SQL 'WHERE', 'AND', and 'OR'
- Aggregation: Equivalents of SQL 'GROUPBY'

In [1]:
import json
import requests
import pandas as pd

# Load secrets from credentials.json
url = 'https://api.foursquare.com/v2/venues/explore'
with open('/Users/gdamico/flatiron/foursquare_seattle-ds-career-040119/.secrets/credentials.json') as f:
    params = json.load(f)

In [2]:
params['v'] = '20190604'
params['ll'] = '47.608, -122.336',
params['query'] = 'chinese',
params['intent'] = 'browse',
params['radius'] = 100000,
params['limit'] = 100

For more on Foursquare's venues API and its parameters, see [here](https://developer.foursquare.com/docs/api/venues/search).

In [3]:
response = requests.get(url=url, params=params)
data = json.loads(response.text)

In [8]:
data

In [9]:
data.keys()

In [10]:
data['response'].keys()

In [11]:
data['meta']

In [12]:
type(data['response'])

In [13]:
data['response']['groups'][0]['items'][0]['venue']

In [4]:
info = []

for store in data['response']['groups'][0]['items']:
    info.append(store['venue'])

foursq_df = pd.DataFrame(info)

In [14]:
foursq_df.head()

In [15]:
type(foursq_df['location'][0])

In [16]:
foursq_df['location'][0]['lat']

## Putting in Mongo

We could do all we need in the terminal, but we can also make use of pymongo, which is a Python package that interfaces with mongo databases!

In [17]:
import pymongo

client = pymongo.MongoClient('mongodb://127.0.0.1:27017')

client.list_database_names()

In [16]:
db = client['foursquare']

In [18]:
db.list_collection_names()

In [18]:
db.create_collection('foursquare_chinese')

In [19]:
db.list_collection_names()

In [ ]:
db['foursquare_chinese'].insert_many(info)

In [20]:
db['foursquare_chinese'].inserted_ids

In [21]:
chinese = db['foursquare_chinese']

In [21]:
chinese.find({})[0]

In [22]:
chinese.find({'name': 'King Noodle'})[0]

### Updating

In [ ]:
liao = {'name': 'Chef Liao'}

In [ ]:
chinese.update_one(liao, {'$set': {'rating': 'five stars'}})

In [26]:
chinese.find({'name': 'Chef Liao'})[0]

### Filtering

We can specify either the keys/values we want displayed (with '1') or the keys/values we do NOT want displayed (with '0').

In [23]:
for eatery in chinese.find({}, {'name': 1, 'location': 1}):
    print(eatery)

The '\_id' key is the only one whose value (0 or 1) can be different from the rest.

In [24]:
for eatery in chinese.find({}, {'_id': 0, 'name': 1, 'location': 1}):
    print(eatery)

### Sorting

In [25]:
chinese.find({}, {'_id': 0, 'name': 1, 'location': 1}).sort('name')[0]

### Aggregating

Try this one yourselves!

## New Foursquare Search! New Mongo Collection!

Adjust the parameters of the API call and collect more data. Then add it as a collection to your Mongo db and practice mongo commands like:

`.find()`
`.delete_one()`
`.update()`
`.update_many()`
`.insert_one()`
`.insert_many()`

For example, try adding a new document to your collection with the following information:

{'name': \[yourname\], 'activitiy': 'was here', 'school': 'flatiron'}

## MongoDB Atlas: MongoDB in the Cloud

MongoDB Atlas is your ticket to MongoDB in the cloud! This may be desirable if you're running into space issues with large databases or if you need to collaborate with others on a project.

Here I'll walk through simple first steps to setting up MongoDB Atlas:

1. Start here: https://www.mongodb.com/cloud/atlas
2. Click on “Start Free”
3. You’ll supply your email, first and last name, and a password at the registration site (https://www.mongodb.com/cloud/atlas/register)
4. Now click “Create a Cluster”
5. Select “Starter Clusters”
6. Configure Cluster
7. Select AWS as provider and “Oregon” as region
8. Leave Cluster Tier as is
9. Leave Additional Settings as is
10. Edit Cluster Name
11. Wait for your Cluster to be built

To connect: <br/>
12. Click on ‘Connect’
13. Whitelist your connection IP address using your existing IP Address
14. Name it: Laptop on WeWork wifi
15. Create a mongoDB User
16. Choose a connection method
17. Click on ‘Connect Your Application’
18. Choose your driver version
19. Driver: Python
20. Version: 3.6 or later
21. Add your connection string into your application code
22. Click on Full Driver Example
23. Replace <password> with the password for the <dbUser> user.

In the terminal, be sure you’re in the learn-env conda environment before typing: <br/>
`conda install pymongo` <br/>
`conda install dnspython`


- To share your database with a team member, follow these steps: <br/>
    Database Access → Click on “Add New User” <br/>
    Assign each team member the following: <br/>
    - A user name
    - A password
    Send your team member the following: <br/>
    Their user name you assigned to them <br/>
    Their password you assigned to them <br/>
    Your mongodb connection string

In [2]:
# Let's do a quick demo of adding data to a cluster on MongoDB Atlas!

import pymongo

In [3]:
#!pip install dnspython

In [4]:
client = pymongo.MongoClient("mongodb+srv://gadamico:g%2CA%2Cd%2C1@\
gregcluster200204-7ckf3.mongodb.net/test?retryWrites=true&w=majority")

In [29]:
client.list_database_names()

In [28]:
db = client.test
db.list_collection_names()

In [27]:
db.people.find({})[0]

In [26]:
import datetime
personDocument = {
  "name": { "first": "Charles", "last": "Babbage" },
  "birth": datetime.datetime(1791, 12, 26),
  "death": datetime.datetime(1871, 10, 18),
  "contribs": [ "computer", "difference engine"]
}

db.people.insert_one(personDocument)